In [1]:
#Install libraries
!pip install geopandas
!pip install geopy
!pip install shapely 
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
#Import Libraries
import pandas as pd
import geopandas as ps
import geopy as gy
import shapely as sy
import dask.dataframe as dd
import pandasql as pq
from shapely.geometry import Point, Polygon
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [3]:
#Setup the path for the file -- might have to change this if yours is named differently
bucket = 'daen-690-pacific-deviations/raw-data' #Bucket name
data_key = 'TOMRDate=2021-12-24.csv' #Path to the CSV file 
data_location = 's3://{}/{}'.format(bucket, data_key)

#Import all of the raw data 
rawData_df = dd.read_csv(data_location, assume_missing=True)
print(f'Total record count : ',len(rawData_df.index))

Total record count :  3303988


In [4]:
def filterAttributes():
    #New dataframe with selected attributes from the raw data
    airspaceData_df = rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong",
                     "FRN145FLFlightLevel", "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                     "FRN161TNTrackNumber"]]

    #Rename columns to make it easier to read
    airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                      'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                      'FRN131HRPWCFloatingPointLong': "Longitude", 
                                                      'FRN145FLFlightLevel': "FlightLevel", 
                                                      'FRN170TITargetId': "TargetID", 
                                                      'RESHSelectedHeading': "SelectedHeading", 
                                                      'FRN80TATargetAddress': "TargetAddress",
                                                      'FRN161TNTrackNumber': "TrackNumber"})
    
    # Remove anything less than 240 flight level 
    airspaceData_df = airspaceData_df[(airspaceData_df['FlightLevel'] >= 240)]
    
    #Change flight level scale to feet (FL1 = 100 ft)
    airspaceData_df['FlightLevel'] = airspaceData_df['FlightLevel'].apply(lambda x: x * 100, meta=('FlightLevel', 'float64'))
    
    airspaceData = airspaceData_df.compute()
    
    return airspaceData

In [5]:
def firstFiveSeconds():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    char = ['T','Z']
    for x in char:
        airspaceData["DateTime"] = airspaceData["DateTime"].str.replace( x ," ")

    # Formatted Datetime
    airspaceData["DateTime"] = pd.to_datetime(airspaceData["DateTime"], format="%Y-%m-%d %H:%M:%S")
    
    # Create 4 new columns for Hour, Minute, Second and Microsecond
    airspaceData["Hour"] = airspaceData["DateTime"].dt.hour
    airspaceData["Minute"] = airspaceData["DateTime"].dt.minute
    airspaceData["Second"] = airspaceData["DateTime"].dt.second
    airspaceData["microSecond"] = airspaceData["DateTime"].dt.microsecond
    
    # Reorder columns
    airspaceData = airspaceData[["DateTime","Hour","Minute","Second","microSecond","Latitude","Longitude","FlightLevel",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]
    
    rawAircraftData = airspaceData
    
    #Keep only records for the first 5 seconds to speed up processing time 
    airspaceData = airspaceData[(airspaceData['Second'] < 5)]
    
    return (rawAircraftData)

In [6]:
def removeHISpace():
    
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    #Coordinates for Hawaii airspace
    v0 = (26.14472222, -158.62194444) 
    v1 = (26.105, -160.63166667)
    v2 = (25.67611111, -161.69111111)
    v3 = (25.05666667, -162.64972222)
    v4 = (24.16889, -163.26638889)
    v5 = (23.25833, -163.855)
    v6 = (22.20555556, -163.91444444)
    v7 = (33.10266389, 130.47177778)
    v8 = (20.11666667, -163.3)
    v9 = (19.65805556,-162.69944444)
    v10 = (19.415, -162.38361111)
    v11 = (18.40777778, -160.81416667)
    v12 = (18.0525, -160.26972222)
    v13 = (17.75583333, -159.53888889)
    v14 = (17.17055556, -157.75666667) 
    v15 = (17.805,-156.06805556)
    v16 = (18.10888889, -155.71166667)
    v17 = (19.14222222, -154.48333333)
    v18 = (19.22293333, -151.87963333)
    v19 = (20.69694444, -151.01916667) 
    v20 = (21.54777778, -151.46638889)
    v21 = (22.34416667,-151.88527778)
    v22 = (23.02416667, -152.57777778)
    v23 = (23.78055556, -153.36611111)
    v24 = (24.29583333, -154.25)
    v25 = (24.72138889, -155.26305556)
    v26 = (25.19583333, -156.42111111)

    # Polygon
    coords = [v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, v17, v18, v19, v20, v21, v22, v23, v24, v25, v26]
    poly = Polygon(coords)
    
    #Sort flights into what is in the airspace and what is not
    hawaiiAir = []
    loc = 0

    while loc < len(airspaceData):
      p1 = Point(airspaceData.iloc[loc][5], airspaceData.iloc[loc][6])
      hawaiiAir.append(p1.within(poly))
      loc = loc + 1

    airspaceData['nearHawaii'] = hawaiiAir
    
    #Filter out only the ones in the airspace
    airspaceData = airspaceData[(airspaceData['nearHawaii'] == False)]
    airspaceData = airspaceData.drop(columns=['nearHawaii'])
        

In [7]:
def aircraftDirection():
    #Set the dataframe that will be altered through this block of code
    global airspaceData
    
    # Replace missing value with -1
    airspaceData['SelectedHeading'] = airspaceData['SelectedHeading'].fillna(-1)
    
    # Assign Direction "E" for 0-180 degree, "W" for 180-360 degree, "NA" is record with null values 
    conditionlist = [
        (airspaceData['SelectedHeading'] < 0) ,
        (airspaceData['SelectedHeading'] >= 0) & (airspaceData['SelectedHeading'] <180),
        (airspaceData['SelectedHeading'] > 180)]
    choicelist = ['NA', 'E', 'W']
    airspaceData['Direction'] = np.select(conditionlist, choicelist)

In [8]:
def filterData():
    
    rawAircraftData = firstFiveSeconds()
    removeHISpace()
    aircraftDirection()
    
    return rawAircraftData

In [9]:
airspaceData = filterAttributes()
allAircraftData = filterData()

/tmp/ipykernel_28813/88026422.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airspaceData['nearHawaii'] = hawaiiAir


In [30]:
def minuteFilter(HourCounter,MinuteCounter):

    global airspaceData

    #create SQL query for flights between the start and end time
    sql1 = "SELECT *, min(Second) FROM airspaceData WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

    #Run query and store results
    recordsInMinute = pq.sqldf(sql1, globals())
    del recordsInMinute['min(Second)']

    return (recordsInMinute)

In [31]:
def lonTableCreate(recordsByMinuteDF):
    #Create tables to check for proximity in latitude
    sql4 = "SELECT * FROM recordsByMinuteDF ORDER BY Longitude"

    #Run query and store results
    LongitudeOrderDF = pq.sqldf(sql4, locals())

    return (LongitudeOrderDF)

In [32]:
def latTableCreate(recordsByMinuteDF):
    #Create tables to check for proximity in latitude
    sql3 = "SELECT * FROM recordsByMinuteDF ORDER BY Latitude"

    #Run query and store results
    LatitudeOrderDF = pq.sqldf(sql3, locals())

    return (LatitudeOrderDF)

In [22]:
# Implement the formula below
def distance_d(point0,pointX):
    # The function "radians" is found in the math module
    LoA = radians(point0[1])  
    LoB = radians(pointX[1])
    LaA=  radians(point0[0])  
    LaB = radians(pointX[0]) 
    # The "Haversine formula" is used.
    D_Lo = LoB - LoA 
    D_La = LaB - LaA 
    P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
    Q = 2 * asin(sqrt(P))   
    # The earth's radius in kilometers.
    R_km = 6371  
 
    # Change the kilometer to  nautical miles
    R_nm = R_km*0.539956803

    # Then we'll compute the outcome.
    return(Q * R_nm)

In [23]:
# Create function to set up boundary within 25 nm by latitude, longitude 
def limit_lon(point0):
    '''
    use with LongitudeOrderDF
    note: distance from point to longitude boundary of each row is around 24.9715
    '''
    LaA = radians(point0[0])
    onedeg_long = cos(LaA)*(69.172*0.868976242)
    add = 25/onedeg_long 
    pointlimit = (point0[0],point0[1]+add)
    return pointlimit[1]

def limit_lat(point0):
    '''
    use with LatitudeOrderDF
    note: distance from point to longitude boundary of each row is 25.016857125339488
    '''
    onedeg_lat = 60 
    add = 25/onedeg_lat
    pointlimit = (point0[0]+add,point0[1])
    return pointlimit[0]

In [41]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','FlightLevel',
                             'TargetID', 'SelectedHeading', 'TargetAddress','Direction']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [25]:
#Calculate the distance of the points closest to each other by longitidue and latitude
def proximityCalc(LongitudeOrderDF, LatitudeOrderDF):
    longpoint_a = []
    longpoint_b = []
    longdistance_ab = []

    latpoint_a = []
    latpoint_b = []
    latdistance_ab = []

    for a in LongitudeOrderDF.index:
        for n in range(1,len(LongitudeOrderDF)):
            b = a+n
            if b < len(LongitudeOrderDF):
                point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
                pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']
                if pointX[1] <= limit_lon(point0): # Check if longitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        longpoint_a.append(a)
                        longpoint_b.append(b)
                        longdistance_ab.append(distance)
                    else:
                        break

    for a in LatitudeOrderDF.index:   
        for n in range(1,len(LatitudeOrderDF)):
            b = a+n
            if b < len(LatitudeOrderDF):
                point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
                pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
                if pointX[0] <= limit_lat(point0):# Check if latitude of pointX is within the boundary
                    distance = distance_d(point0,pointX)
                    if distance <= 25: # Check distance within 25 nm
                        latpoint_a.append(a)
                        latpoint_b.append(b)
                        latdistance_ab.append(distance)
                    else:
                        break
        
    # Apply function to select and merge data frame
    LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
    LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

    # Concatenate results from order by longitude and latitude
    Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

    # Delete duplicate pairs of TargetID x and y regardless of order
    Resultsdf['list_target'] = Resultsdf.apply(lambda row: tuple(sorted([row['TargetID_x']]+[row['TargetID_y']])), axis = 1)
    ResultsDF = Resultsdf.drop_duplicates(subset = 'list_target',keep = 'first').reset_index(drop = True)
    ResultsDF.drop('list_target', axis=1, inplace=True)

    return (ResultsDF)

In [28]:
def distanceCalc(resultsDF):
    heightDifference = []
    potentialLoss1000 = []
    potentialLoss400 = []

    counter = 0

    while counter < len(resultsDF):
        difference = abs((resultsDF['FlightLevel_x'][counter]) - (resultsDF['FlightLevel_y'][counter]))
        heightDifference.append(difference)

        if difference <= 1000:
            potentialLoss1000.append('True')
            if difference <= 400:
                potentialLoss400.append('True')
            else:
                potentialLoss400.append('False')
        else:
            potentialLoss1000.append('False')
            potentialLoss400.append('False')

        counter = counter + 1

    resultsDF['HeightDifference_ft'] = heightDifference
    resultsDF['potentialLoss400'] = potentialLoss400
    resultsDF['potentialLoss1000'] = potentialLoss1000

    return (resultsDF)

In [39]:
HourCounter = 0
finalResults = pd.DataFrame()

while HourCounter < 1:
    #Create table for the minute
    for MinuteCounter in range(0,60):
        #Create table for the minute
        recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)
    
        #Create longitude and latitude table for proximity analysis
        LongitudeOrderDF = lonTableCreate(recordsByMinuteDF)
        LatitudeOrderDF = latTableCreate(recordsByMinuteDF)
    
        #calculate proximity
        resultsDF = proximityCalc(LongitudeOrderDF, LatitudeOrderDF)
        if resultsDF.empty == True:
            # if the results dataframe is empty, then break out of for-loop
            break
        else:
            #Calculate distance
            resultsDF = distanceCalc(resultsDF)
            #Add the results for this minute to the overall results 
            finalResults = pd.concat([finalResults, resultsDF], ignore_index=True)
    HourCounter = HourCounter + 1

In [40]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM finalResults WHERE potentialLoss400 = 'True' "

#Run query and store results
LossCandidates400 = pq.sqldf(sql6, locals())

print(LossCandidates400)


                   DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0  2021-12-24 00:24:02.047000       0        24         2          47000   
1  2021-12-24 00:45:00.875000       0        45         0         875000   

   Latitude_x  Longitude_x  FlightLevel_x TargetID_x  SelectedHeading_x  ...  \
0   22.278488  -155.472823        29800.0     AAL432          78.046875  ...   
1   21.387222  -155.755821        30575.0    SWA1385         196.875000  ...   

  Longitude_y FlightLevel_y TargetID_y  SelectedHeading_y  TargetAddress_y  \
0 -155.816895       30000.0     DAL495          45.703125           A789BC   
1 -155.441362       30475.0    SWA1310          68.203125           ABFD8C   

   Direction_y   Distance  HeightDifference_ft  potentialLoss400  \
0            E  19.153653                200.0              True   
1            E  24.140765                100.0              True   

   potentialLoss1000  
0               True  
1               True  

[2 rows x 28 columns

----- NEW CODE FOR TESTING VISUALIZATIONS -----

In [44]:
#Convert pandas dataframe to a CSV to export
LossCandidates400.to_csv('LossCandidates400.csv', index=False, header=True)

#Moving the CSV file of the pandas dataframe into the S3 bucket to download and move to other tools to see what works best
import sagemaker, boto3, os
bucket = sagemaker.Session().default_bucket()
prefix = "demo-sagemaker-filtered-data"

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/LossCandidates400.csv')).upload_file('LossCandidates400.csv')

In [24]:
#Get the points of interest

deviation1 = allAircraftData.loc[((allAircraftData['TargetID'] == "AAL432") | (allAircraftData['TargetID'] == "DAL495")) & ((allAircraftData['Minute'] >= 21) & (allAircraftData['Minute'] <= 27))]

deviation2 = allAircraftData.loc[((allAircraftData['TargetID'] == "SWA1385") | (allAircraftData['TargetID'] == "SWA1310")) & ((allAircraftData['Minute'] >= 42) & (allAircraftData['Minute'] <= 48))]


In [27]:
#Convert pandas dataframe to a CSV to export
deviation1.to_csv('deviation1.csv', index=False, header=True)

#Moving the CSV file of the pandas dataframe into the S3 bucket to download and move to other tools to see what works best
import sagemaker, boto3, os
bucket = sagemaker.Session().default_bucket()
prefix = "demo-sagemaker-filtered-data"

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/deviation1.csv')).upload_file('deviation1.csv')

In [28]:
#Convert pandas dataframe to a CSV to export
deviation2.to_csv('deviation2.csv', index=False, header=True)

#Moving the CSV file of the pandas dataframe into the S3 bucket to download and move to other tools to see what works best
import sagemaker, boto3, os
bucket = sagemaker.Session().default_bucket()
prefix = "demo-sagemaker-filtered-data"

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/deviation2.csv')).upload_file('deviation2.csv')